# VRP && VRPPD in ortools 

- 第一个代码块是VRP的部分；
- 第二个代码块是VRPPD的部分；来自 [ortools official](https://developers.google.cn/optimization/routing/)

In [12]:
"""Simple Vehicles Routing Problem (VRP).

   This is a sample using the routing library python wrapper to solve a VRP
   problem.
   A description of the problem can be found here:
   http://en.wikipedia.org/wiki/Vehicle_routing_problem.

   Distances are in meters.
"""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["num_vehicles"] = 4
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print("No solution found !")


if __name__ == "__main__":
    main()

Objective: 177500
Route for vehicle 0:
 0 ->  9 ->  10 ->  2 ->  6 ->  5 -> 0
Distance of the route: 1712m

Route for vehicle 1:
 0 ->  16 ->  14 ->  8 -> 0
Distance of the route: 1484m

Route for vehicle 2:
 0 ->  7 ->  1 ->  4 ->  3 -> 0
Distance of the route: 1552m

Route for vehicle 3:
 0 ->  13 ->  15 ->  11 ->  12 -> 0
Distance of the route: 1552m

Maximum of the route distances: 1712m


In [30]:
import random

def generate_rand(nodes):
    random.seed(124)
    dist = [[0 for _ in range(nodes)] for _ in range(nodes)]
    for i in range(nodes):
        for j in range(i + 1, nodes):
            dist[i][j] = random.randint(300, 1000)
            dist[j][i] = dist[i][j] 

    return dist 

if __name__ == "__main__":
    nodes = 101
    dist = generate_rand(nodes)
    for d in dist:
        print(d)
    incidence = [[i, i + nodes // 2] for i in range(1, nodes // 2 + 1)]
    print(incidence)

[0, 584, 861, 305, 481, 360, 726, 586, 672, 332, 624, 469, 706, 618, 731, 322, 856, 785, 931, 735, 318, 534, 523, 403, 712, 623, 513, 445, 697, 495, 964, 664, 850, 602, 708, 474, 408, 765, 825, 919, 978, 414, 694, 882, 847, 790, 412, 921, 755, 392, 435, 845, 693, 554, 488, 766, 824, 339, 962, 758, 952, 714, 953, 660, 905, 773, 381, 477, 364, 843, 363, 359, 540, 845, 837, 403, 930, 781, 783, 920, 783, 382, 893, 792, 956, 960, 329, 781, 543, 711, 1000, 462, 524, 321, 620, 826, 665, 783, 747, 458, 785]
[584, 0, 813, 625, 676, 374, 922, 356, 557, 700, 390, 415, 700, 510, 815, 929, 506, 734, 954, 529, 372, 384, 332, 335, 466, 412, 882, 475, 394, 512, 507, 473, 337, 720, 791, 782, 301, 966, 354, 390, 775, 664, 394, 375, 845, 738, 729, 482, 669, 524, 561, 872, 747, 537, 749, 317, 741, 944, 835, 994, 835, 861, 664, 543, 650, 752, 822, 438, 821, 948, 678, 633, 380, 427, 763, 388, 898, 933, 622, 627, 594, 806, 446, 365, 834, 313, 972, 758, 682, 920, 470, 393, 641, 389, 754, 345, 944, 569, 990, 8

In [31]:
"""Simple Pickup Delivery Problem (PDP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(dist, incidence):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = dist
    # data["distance_matrix"] = [
    #     # fmt: off 邻接矩阵
    #   [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
    #   [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
    #   [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
    #   [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
    #   [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
    #   [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
    #   [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
    #   [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
    #   [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
    #   [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
    #   [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
    #   [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
    #   [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
    #   [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
    #   [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
    #   [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
    #   [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
    #     # fmt: on
    # ]
    data["pickups_deliveries"] = incidence
    data["num_vehicles"] = 4
    # 车辆数
    data["depot"] = 0
    # depot ID
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        total_distance += route_distance
    print(f"Total Distance of all routes: {total_distance}m")


def main(dist, incidence):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(dist, incidence)

    # Create the routing index manager.
    # RoutingIndexManager 管理网络中的节点、车辆的情况： *args: 节点个数、车辆个数、出发点
    
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    # 创建路线规划模型
    routing = pywrapcp.RoutingModel(manager)


    # Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        
        # IndexToNode 功能解释：
        # TODO:
        # 索引是求解器对回调的内部引用。把传入的节点在求解器内部给定一个索引。
        
        
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    
    # ResisterTransiteCallback
    # ???

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        25000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    
    # 这里的dimension概念是ortools中很重要的：https://developers.google.cn/optimization/routing/dimensions
    # 
    # 求解器使用dimension对象来跟踪沿车辆路线累积的数量，例如行驶时间，或者如果车辆正在取货和送货，追踪其运载的总重量。
    # 如果VRP涉及此类数量，无论是在约束还是目标函数中，都需要定义一个维度来指定它们。
    
    
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    
    # 如果没有dimension_name 的维度，那么就Die（退出）
    
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    
    # TODO：这一部分没有怎么看得懂，这个Span Cost Coefficeient 具体是指的啥？
    # 似乎是影响目标函数是否需要乘一个常数？Official statement 如下
    # The method SetGlobalSpanCostCoefficient sets a large coefficient (100) for the global span of the routes,
    # which in this example is the maximum of the distances of the routes. 
    # This makes the global span the predominant factor in the objective function, 
    # so the program minimizes the length of the longest route.
    
    

    # Define Transportation Requests.
    for request in data["pickups_deliveries"]:
        
        # FIXME: ortools 封装好了路线规划中带取送货的情况，只需要在模型设置好了之后设置起终点即可。
        
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index)
        )
        
        # 这里约束了：必须是要同一个车配送VehicleCar必须是相同的
        
        routing.solver().Add(
            distance_dimension.CumulVar(pickup_index)
            <= distance_dimension.CumulVar(delivery_index)
        )
        # 这一句话就加了一个约束：
        # 因为pickup一定要在deliver 之前进行，所以累积到 pickup 的行驶距离（也就是前面声明的维度）：
        # 一定要不小于 deliver 点的行驶距离

        # CumulVar 的解释：Get the cumul, transit and slack variables for the given node (given as int64 var index).
        # 这里的pickop_index 和delivery_index 必须要限制为int整数类型

    # Setting first solution heuristic.
    # 选择搜索策略
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main(dist, incidence)

Objective: 1002817
Route for vehicle 0:
 0 ->  22 ->  25 ->  31 ->  27 ->  10 ->  19 ->  72 ->  3 ->  41 ->  77 ->  39 ->  60 ->  24 ->  44 ->  89 ->  32 ->  7 ->  74 ->  82 ->  81 ->  91 ->  69 ->  75 ->  53 ->  94 ->  57 -> 0
Distance of the route: 9636m

Route for vehicle 1:
 0 ->  20 ->  1 ->  43 ->  47 ->  2 ->  51 ->  26 ->  30 ->  76 ->  12 ->  50 ->  42 ->  62 ->  52 ->  100 ->  93 ->  4 ->  97 ->  92 ->  18 ->  54 ->  80 ->  68 ->  70 -> 0
Distance of the route: 9382m

Route for vehicle 2:
 0 ->  15 ->  37 ->  36 ->  5 ->  8 ->  35 ->  46 ->  23 ->  6 ->  56 ->  48 ->  73 ->  58 ->  87 ->  85 ->  16 ->  98 ->  11 ->  21 ->  65 ->  96 ->  71 ->  55 ->  61 ->  86 ->  66 -> 0
Distance of the route: 9648m

Route for vehicle 3:
 0 ->  49 ->  14 ->  33 ->  99 ->  29 ->  28 ->  34 ->  79 ->  64 ->  84 ->  40 ->  17 ->  45 ->  95 ->  90 ->  67 ->  9 ->  38 ->  83 ->  78 ->  13 ->  63 ->  59 ->  88 -> 0
Distance of the route: 9351m

Total Distance of all routes: 38017m


## 看代码启示

1. 写代码的时候传入数据尽量直接按照邻接矩阵等输入；

> 提问：别的输入格式是否也可以呢？比如边；

2. Manage 中的IndexToNode，基于输入的数据，在求解器内给每个点设置一个索引方便计算；

3. 专门写一个用于输出结果的打印函数（print_solution）

4. “维度”的概念：dimension

5. 把返回节点之间距离的写成回调函数的形式；